# Ridge and Lasso regression 

We use Ridge and Lasso regression to predict the GDP.


In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

In [52]:
data = pd.read_csv("../Dataset/FRED_quarterly_stat.csv")
print(data.shape)
data.head()
# data are already stationary

(204, 178)


,sasdate,GDPC1,PCECC96,PCDGx,PCESVx,PCNDx,GPDIC1,FPIx,Y033RC1Q027SBEAx,PNFIx,...,UEMPMEAN,CES0600000007,GS5,HWIURATIOx,ISRATIOx,CONSPIx,CP3M,TNWMVBSNNCBBDIx,TNWBSNNBBDIx,S&P div yield
0,06/01/1968,0.719628,0.656335,0.915296,0.669177,0.530723,1.630199,-0.106120,-1.063844,-0.536419,...,-0.6000,0.1667,0.2300,0.0830,0.0069,-0.0009,0.50,-67.34,-53.81,-0.1733
1,09/01/1968,0.335269,0.806416,1.911046,0.548293,0.686066,-1.200849,0.670529,1.059441,0.591725,...,0.0333,0.0333,-0.3333,0.0629,0.0034,-0.0007,-0.12,-7.52,-8.88,-0.0179
2,12/01/1968,0.170474,0.195462,-0.255424,0.473817,0.010125,0.469887,1.015234,1.483582,1.168323,...,-0.0667,-0.0667,0.2500,0.1212,-0.0172,0.0010,0.00,16.89,16.98,-0.1152
3,03/01/1969,0.674453,0.478903,0.814686,0.378184,0.479726,2.484897,1.194080,1.556066,1.097820,...,-0.2666,0.0000,0.5566,0.0401,0.0069,0.0008,0.70,55.56,31.62,0.1570
4,06/01/1969,0.131828,0.279262,-0.114237,0.533750,0.096236,-0.256607,0.231719,0.452702,0.470709,...,-0.1334,0.1334,0.1967,-0.0224,0.0172,0.0000,0.88,23.34,7.59,0.0081


In [53]:
# check if there are nans
data.isnull().sum()

sasdate             1
GDPC1               0
PCECC96             0
PCDGx               0
PCESVx              0
PCNDx               0
GPDIC1              0
FPIx                0
Y033RC1Q027SBEAx    0
PNFIx               0
PRFIx               0
GCEC1               0
FGRECPTx            0
SLCEx               0
EXPGSC1             0
IMPGSC1             0
DPIC96              0
OUTNFB              0
OUTBS               0
INDPRO              0
IPFINAL             0
IPCONGD             0
IPMAT               0
IPDMAT              0
IPNMAT              0
IPDCONGD            0
IPB51110SQ          0
IPNCONGD            0
IPBUSEQ             0
IPB51220SQ          0
                   ..
CIVPART             1
UNRATE              1
UNRATESTx           1
UNRATELTx           1
LNS14000012         1
LNS14000025         1
LNS14000026         1
AWHNONAG            1
AWOTMAN             1
FEDFUNDS            1
TB3MS               1
TB6MS               1
GS1                 1
GS10                1
AAA       

In [54]:
# There are not many NaN so we can drop them
data = data.dropna()

In [55]:
data.head()

,sasdate,GDPC1,PCECC96,PCDGx,PCESVx,PCNDx,GPDIC1,FPIx,Y033RC1Q027SBEAx,PNFIx,...,UEMPMEAN,CES0600000007,GS5,HWIURATIOx,ISRATIOx,CONSPIx,CP3M,TNWMVBSNNCBBDIx,TNWBSNNBBDIx,S&P div yield
0,06/01/1968,0.719628,0.656335,0.915296,0.669177,0.530723,1.630199,-0.106120,-1.063844,-0.536419,...,-0.6000,0.1667,0.2300,0.0830,0.0069,-0.0009,0.50,-67.34,-53.81,-0.1733
1,09/01/1968,0.335269,0.806416,1.911046,0.548293,0.686066,-1.200849,0.670529,1.059441,0.591725,...,0.0333,0.0333,-0.3333,0.0629,0.0034,-0.0007,-0.12,-7.52,-8.88,-0.0179
2,12/01/1968,0.170474,0.195462,-0.255424,0.473817,0.010125,0.469887,1.015234,1.483582,1.168323,...,-0.0667,-0.0667,0.2500,0.1212,-0.0172,0.0010,0.00,16.89,16.98,-0.1152
3,03/01/1969,0.674453,0.478903,0.814686,0.378184,0.479726,2.484897,1.194080,1.556066,1.097820,...,-0.2666,0.0000,0.5566,0.0401,0.0069,0.0008,0.70,55.56,31.62,0.1570
4,06/01/1969,0.131828,0.279262,-0.114237,0.533750,0.096236,-0.256607,0.231719,0.452702,0.470709,...,-0.1334,0.1334,0.1967,-0.0224,0.0172,0.0000,0.88,23.34,7.59,0.0081


In [56]:
# select the data 
dates = data.sasdate
y = data.GDPC1
X = data.drop(['sasdate','GDPC1'], axis =1)

In [57]:
# standardize the data
scaler = StandardScaler()
scaler.fit(X)
X_std = scaler.transform(X)

y_tmp = np.expand_dims(np.array(y), axis = 1)
scaler.fit(y_tmp.reshape(-1,1))
y_std = scaler.transform(y_tmp)

In [58]:
# when we split the data we need to take into consideration the longitudinal feature of the data.
tscv = TimeSeriesSplit(n_splits=5)
n_alphas = 100
alphas = np.linspace(0.01,0.99,100)

coefs_lm = np.zeros((n_alphas,  X.shape[1], 5))
coefs_lasso = np.zeros((n_alphas, X.shape[1], 5))
coefs_ridge = np.zeros((n_alphas,  X.shape[1], 5))

mse_lm_insample = np.zeros((n_alphas, 5))
mse_ridge_insample = np.zeros((n_alphas, 5))
mse_lasso_insample = np.zeros((n_alphas, 5))

mse_lm = np.zeros((n_alphas, 5))
mse_ridge = np.zeros((n_alphas, 5))
mse_lasso = np.zeros((n_alphas, 5))

a_i = 0
for a in alphas:
    ridge = Ridge(alpha = a)
    lasso = Lasso(alpha = a)
    lm = LinearRegression()
    
    # mse_lm_i = []   # it does not depend on alpha! because we don't have any hyperparamet to est in this case
    # mse_ridge_i = []
    # mse_lasso_i = []
    
    i_sub = 0  #sub_set index
    for train_index, test_index in tscv.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # fit the model
        lm.fit(X_train, y_train)
        ridge.fit(X_train, y_train)
        lasso.fit(X_train, y_train)
        # coefs_ridge.append(ridge.coef_)
        
        coefs_lm[a_i,:, i_sub] = lm.coef_
        coefs_ridge[a_i, :, i_sub] = ridge.coef_
        coefs_lasso[a_i, :, i_sub] = lasso.coef_
        
        # in sample predict
        lm_pred_insample = lm.predict(X_train)
        ridge_pred_insample = ridge.predict(X_train)
        lasso_pred_insample = lasso.predict(X_train)
        
        mse_lm_insample[a_i, i_sub] =np.mean((y_train - lm_pred_insample)**2)
        mse_ridge_insample[a_i, i_sub] =np.mean((y_train - ridge_pred_insample)**2)
        mse_lasso_insample[a_i, i_sub] =np.mean((y_train - lasso_pred_insample)**2)
        # mse_lm_i_insample.append(np.mean((y_train - lm_pred_insample)**2))
        # mse_ridge_i_insample.append(np.mean((y_train - ridge_pred_insample)**2))
        # mse_lasso_i_insample.append(np.mean((y_train - lasso_pred_insample)**2))
        
        
        # out of sample predict
        lm_pred = lm.predict(X_test)
        ridge_pred = ridge.predict(X_test)
        lasso_pred = lasso.predict(X_test)
        
        mse_lm[a_i, i_sub] =np.mean((y_test - lm_pred)**2)
        mse_ridge[a_i, i_sub] =np.mean((y_test - ridge_pred)**2)
        mse_lasso[a_i, i_sub] =np.mean((y_test - lasso_pred)**2)
        # mse_lm_i.append(np.mean((y_test - lm_pred)**2))
        # mse_ridge_i.append(np.mean((y_test - ridge_pred)**2))
        # mse_lasso_i.append(np.mean((y_test - lasso_pred)**2))
        
        i_sub += 1
    # mse_lm.append(np.mean(mse_lm_i))
    # mse_ridge.append(np.mean(mse_ridge_i))
    # mse_lasso.append(np.mean(mse_lasso_i))
    a_i += 1

/Users/paoloandreini/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [59]:

print('Mean MSE in oos Linear Regression: ' , np.mean(mse_lm))
print('Mean MSE in oos Ridge: ' , np.mean(mse_ridge))
print('Mean MSE in oos Lasso: ' , np.mean(mse_lasso))

print('Mean MSE insample Linear Regression: ' ,np.mean(mse_lm_insample))
print('Mean MSE insample Ridge Regression: ' ,np.mean(mse_ridge_insample))
print('Mean MSE insample Lasso Regression: ' ,np.mean(mse_lasso_insample))

Mean MSE in oos Linear Regression:  0.04673196026875671
Mean MSE in oos Ridge:  0.03748654029552874
Mean MSE in oos Lasso:  0.07975934919859294
Mean MSE insample Linear Regression:  2.8853472492877507e-22
Mean MSE insample Ridge Regression:  4.6778837407739226e-05
Mean MSE insample Lasso Regression:  0.0784005384842195


In [60]:
# Minimum Values
print('MSE in oos Linear Regression: ' , np.amin(mse_lm))
print('MSE in oos Ridge: ' , np.amin(mse_ridge))
print('MSE in oos Lasso: ' , np.amin(mse_lasso))

print('MSE insample Linear Regression: ' ,np.amin(mse_lm_insample))
print('MSE insample Ridge Regression: ' ,np.amin(mse_ridge_insample))
print('MSE insample Lasso Regression: ' ,np.amin(mse_lasso_insample))

MSE in oos Linear Regression:  0.006560755257083308
MSE in oos Ridge:  0.002041768078033285
MSE in oos Lasso:  0.005435158130952146
MSE insample Linear Regression:  3.484627954551221e-26
MSE insample Ridge Regression:  8.5813385756369e-10
MSE insample Lasso Regression:  0.0016899568948287379


In [61]:
# Minimum Values
print('Optimal λ in Ridge: ' , np.round(alphas[np.argmin(mse_ridge)], 4))
print('Optimal λ in Lasso: ' , np.round(alphas[np.argmin(mse_lasso)], 4))

Optimal λ in Ridge:  0.4456
Optimal λ in Lasso:  0.0496
